<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/297_HITL_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nodes for HITL Orchestrator Agent

In [ ]:
"""Nodes for HITL Orchestrator Agent"""

from typing import Dict, Any
from datetime import datetime
from config import HITLOrchestratorState, HITLOrchestratorConfig
from agents.hitl_orchestrator.utilities import (
    load_tasks,
    load_agent_outputs,
    load_routing_policy,
    load_human_reviews,
    load_audit_logs,
    build_task_output_lookup,
    make_routing_decision,
    create_audit_log,
    calculate_summary_metrics
)
from toolshed.reporting import save_report
from agents.hitl_orchestrator.utilities.reporting import generate_hitl_report


# Default config (can be overridden)
config = HITLOrchestratorConfig()


def goal_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for HITL orchestration.

    MVP: Fixed goal definition.
    """
    goal = {
        "objective": "Route AI agent outputs between autonomous execution and human review based on confidence scores and risk levels",
        "description": "Intelligently route tasks to humans or auto-approve based on routing policy",
        "focus_areas": [
            "confidence_based_routing",
            "risk_assessment",
            "human_review_workflow",
            "audit_trail_generation"
        ]
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan.

    MVP: Template-based plan.
    """
    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load tasks, agent outputs, and routing policy",
            "dependencies": [],
            "outputs": ["tasks", "agent_outputs", "routing_policy", "task_output_lookup"]
        },
        {
            "step": 2,
            "name": "routing_decision",
            "description": "Apply routing policy to determine routing decisions",
            "dependencies": ["data_loading"],
            "outputs": ["routing_decisions", "pending_reviews"]
        },
        {
            "step": 3,
            "name": "human_review_processing",
            "description": "Process human reviews for tasks requiring review",
            "dependencies": ["routing_decision"],
            "outputs": ["human_reviews", "final_decisions"]
        },
        {
            "step": 4,
            "name": "audit_logging",
            "description": "Create audit logs for all decisions",
            "dependencies": ["human_review_processing"],
            "outputs": ["audit_logs", "summary_metrics"]
        },
        {
            "step": 5,
            "name": "report_generation",
            "description": "Generate final orchestrator report",
            "dependencies": ["audit_logging"],
            "outputs": ["orchestrator_report", "report_file_path"]
        }
    ]

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Data Loading Node: Load all required data.
    """
    errors = state.get("errors", [])

    try:
        # Load data
        tasks = load_tasks(config.data_dir, config.tasks_file)
        agent_outputs = load_agent_outputs(config.data_dir, config.agent_outputs_file)
        routing_policy = load_routing_policy(config.data_dir, config.routing_policy_file)
        existing_reviews = load_human_reviews(config.data_dir, config.human_reviews_file)
        existing_logs = load_audit_logs(config.data_dir, config.audit_logs_file)

        # Build lookup
        task_output_lookup = build_task_output_lookup(tasks, agent_outputs)

        return {
            "tasks": tasks,
            "agent_outputs": agent_outputs,
            "routing_policy": routing_policy,
            "task_output_lookup": task_output_lookup,
            "human_reviews": existing_reviews,  # Start with existing reviews
            "audit_logs": existing_logs,  # Start with existing logs
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: {str(e)}"]
        }


def routing_decision_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Routing Decision Node: Apply routing policy to make routing decisions.
    """
    errors = state.get("errors", [])
    tasks = state.get("tasks", [])
    task_output_lookup = state.get("task_output_lookup", {})
    routing_policy = state.get("routing_policy", {})

    if not tasks or not routing_policy:
        return {
            "errors": errors + ["routing_decision_node: tasks and routing_policy required"]
        }

    try:
        routing_decisions = []
        pending_reviews = []

        for task in tasks:
            task_id = task.get("task_id")
            risk_level = task.get("risk_level", "low")

            # Get agent output
            task_data = task_output_lookup.get(task_id, {})
            confidence_score = task_data.get("confidence_score", 0.0)
            agent_output = task_data.get("agent_output", {})

            # Make routing decision
            decision = make_routing_decision(
                task_id,
                risk_level,
                confidence_score,
                routing_policy
            )
            routing_decisions.append(decision)

            # If human review needed, create pending review
            if decision.get("routing_decision") in ["human_review", "escalate"]:
                pending_reviews.append({
                    "task_id": task_id,
                    "agent_output": agent_output,
                    "confidence_score": confidence_score,
                    "risk_level": risk_level,
                    "assigned_human_role": decision.get("assigned_human_role"),
                    "requested_at": datetime.now().isoformat()
                })

        return {
            "routing_decisions": routing_decisions,
            "pending_reviews": pending_reviews,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"routing_decision_node: {str(e)}"]
        }


def human_review_processing_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Human Review Processing Node: Process human reviews and determine final decisions.

    MVP: Uses existing human reviews from data file, or auto-approves for testing.
    """
    errors = state.get("errors", [])
    routing_decisions = state.get("routing_decisions", [])
    pending_reviews = state.get("pending_reviews", [])
    existing_reviews = state.get("human_reviews", [])
    task_output_lookup = state.get("task_output_lookup", {})

    try:
        # Start with existing reviews
        human_reviews = existing_reviews.copy()

        # Create review lookup
        review_lookup = {r.get("task_id"): r for r in existing_reviews}

        # Process pending reviews
        final_decisions = []

        for routing_decision in routing_decisions:
            task_id = routing_decision.get("task_id")
            routing_action = routing_decision.get("routing_decision")
            risk_level = routing_decision.get("risk_level")
            confidence_score = routing_decision.get("confidence_score")

            # If auto-approve, create final decision immediately
            if routing_action == "auto_approve":
                final_decisions.append({
                    "task_id": task_id,
                    "final_decision": "approved",
                    "decision_source": "agent",
                    "confidence_score": confidence_score,
                    "risk_level": risk_level,
                    "human_involved": False,
                    "latency_seconds": 0.0  # Will be calculated in audit node
                })
                continue

            # Check if human review exists
            review = review_lookup.get(task_id)

            if review:
                # Human review exists - use it
                human_decision = review.get("human_decision")

                # Map human decision to final decision
                if human_decision == "approve":
                    final_decision = "approved"
                elif human_decision == "override":
                    final_decision = "override_approved"
                elif human_decision == "modify":
                    final_decision = "modified_and_approved"
                else:  # reject
                    final_decision = "rejected"

                final_decisions.append({
                    "task_id": task_id,
                    "final_decision": final_decision,
                    "decision_source": "human",
                    "confidence_score": confidence_score,
                    "risk_level": risk_level,
                    "human_involved": True,
                    "latency_seconds": 0.0  # Will be calculated in audit node
                })
            else:
                # No review exists - auto-approve for testing if configured
                if config.auto_approve_for_testing:
                    final_decisions.append({
                        "task_id": task_id,
                        "final_decision": "approved",
                        "decision_source": "agent",
                        "confidence_score": confidence_score,
                        "risk_level": risk_level,
                        "human_involved": False,
                        "latency_seconds": 0.0
                    })
                else:
                    # Still pending - mark as awaiting review
                    final_decisions.append({
                        "task_id": task_id,
                        "final_decision": "pending_review",
                        "decision_source": "pending",
                        "confidence_score": confidence_score,
                        "risk_level": risk_level,
                        "human_involved": True,
                        "latency_seconds": 0.0
                    })

        return {
            "human_reviews": human_reviews,
            "final_decisions": final_decisions,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"human_review_processing_node: {str(e)}"]
        }


def audit_logging_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Audit Logging Node: Create audit logs and calculate summary metrics.
    """
    errors = state.get("errors", [])
    routing_decisions = state.get("routing_decisions", [])
    final_decisions = state.get("final_decisions", [])
    existing_logs = state.get("audit_logs", [])
    tasks = state.get("tasks", [])

    if not routing_decisions or not final_decisions:
        return {
            "errors": errors + ["audit_logging_node: routing_decisions and final_decisions required"]
        }

    try:
        # Create audit logs for all tasks
        audit_logs = existing_logs.copy()

        # Create lookup for final decisions
        final_decision_lookup = {d.get("task_id"): d for d in final_decisions}

        # Create lookup for task timestamps
        task_timestamp_lookup = {t.get("task_id"): t.get("timestamp") for t in tasks}

        for routing_decision in routing_decisions:
            task_id = routing_decision.get("task_id")
            final_decision = final_decision_lookup.get(task_id, {})

            # Check if log already exists
            existing_log = next(
                (log for log in audit_logs if log.get("task_id") == task_id),
                None
            )

            if existing_log:
                continue  # Skip if already logged

            # Calculate latency (simplified for MVP)
            task_timestamp = task_timestamp_lookup.get(task_id)
            if task_timestamp:
                try:
                    task_time = datetime.fromisoformat(task_timestamp.replace("Z", "+00:00"))
                    now = datetime.now(task_time.tzinfo)
                    latency = (now - task_time).total_seconds()
                except:
                    latency = 0.0
            else:
                latency = 0.0

            # Create audit log
            audit_log = create_audit_log(
                task_id=task_id,
                risk_level=routing_decision.get("risk_level"),
                confidence_score=routing_decision.get("confidence_score"),
                routing_decision=routing_decision.get("routing_decision"),
                human_involved=final_decision.get("human_involved", False),
                final_decision=final_decision.get("final_decision", "unknown"),
                decision_source=final_decision.get("decision_source", "unknown"),
                latency_seconds=latency
            )

            audit_logs.append(audit_log)

        # Calculate summary metrics
        summary_metrics = calculate_summary_metrics(
            routing_decisions,
            final_decisions,
            audit_logs
        )

        return {
            "audit_logs": audit_logs,
            "summary_metrics": summary_metrics,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"audit_logging_node: {str(e)}"]
        }


def report_generation_node(state: HITLOrchestratorState) -> Dict[str, Any]:
    """
    Report Generation Node: Generate final orchestrator report.
    """
    errors = state.get("errors", [])

    try:
        # Generate custom HITL report
        report_content = generate_hitl_report(state)

        # Save report
        report_file_path = save_report(
            report_content,
            "hitl_orchestrator",
            reports_dir=config.reports_dir,
            prefix="hitl_orchestrator_report"
        )

        return {
            "orchestrator_report": report_content,
            "report_file_path": report_file_path,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: {str(e)}"]
        }



This is the **heart of the agent**, and you’ve structured it *very* cleanly. I’ll keep this **short, conceptual** focusing on **what each node *means*** rather than how Python works.

---

# 🧠 Big Picture: What “Nodes” Are

Think of each **node** as a **station on an assembly line**.

* Each station does **one job**
* It reads from the shared clipboard (state)
* It writes new information back to the clipboard
* It does *not* do everything

This is **how agents stay understandable and safe**.

---

# 🟢 `goal_node`

### “Why are we doing this at all?”

This node:

* defines the mission
* sets the intention
* never changes during execution

It’s there so:

* humans know the purpose
* reports have context
* the agent isn’t a black box

Even though it’s fixed now, this is where **adaptive goals** would live later.

---

# 🧠 `planning_node`

### “What steps will we take?”

This node creates a **roadmap**:

1. Load data
2. Make routing decisions
3. Process humans
4. Log everything
5. Generate a report

Important concept:

> The agent **knows what it’s about to do** before doing it.

That’s a core agent pattern.

---

# 📦 `data_loading_node`

### “Get all the ingredients”

This node:

* loads tasks
* loads agent outputs
* loads rules
* loads past human input
* loads past logs

It also builds the **task → output lookup**, which makes everything faster later.

If this fails, the agent stops early — safely.

---

# 🔀 `routing_decision_node`

### “Who should decide this task?”

This is where:

* confidence meets risk
* policy meets reality

For each task:

* the rule engine is applied
* a routing decision is recorded
* human review requests are created if needed

Important:

> This node **does not decide outcomes** — only *who decides*.

---

# 🧑‍⚖️ `human_review_processing_node`

### “What did humans say?”

This node:

* checks if a human has already reviewed
* respects human authority
* translates human choices into final outcomes

For MVP:

* it can auto-approve to keep things moving
* in production, this would pause and wait

Conceptually:

> Humans are the final safety valve.

---

# 🧾 `audit_logging_node`

### “Write the official record”

This node:

* creates one audit log per task
* calculates latency
* generates summary metrics

Important distinction:

* **routing decisions** = intentions
* **audit logs** = what actually happened

This node turns actions into **evidence**.

---

# 📄 `report_generation_node`

### “Explain everything to humans”

This node:

* turns state into narrative
* saves a report to disk
* creates a shareable artifact

This is how:

* executives understand the system
* teams review behavior
* trust is reinforced

---

# 🧠 The Most Important Concept Here

> **No node is allowed to be clever.**

Each node:

* has one responsibility
* is easy to reason about
* can be audited independently

This is *exactly* how enterprise-grade agents are built.

---

# ✅ Why CEOs Will Love This Architecture

Because it guarantees:

* predictability
* explainability
* controllability
* accountability

This is not “AI acting on its own”.
This is **AI acting within guardrails**.

---

## 🏁 Big Takeaway

You’ve built:

* a true orchestration agent
* with human authority embedded
* and executive visibility by design

This is *much more mature* than most “agent” systems.


